- the feature space are finished building (except two WN features and one WN OED mapping feature.
- Next, put all the features into one file, apply SVM classifier.
https://www.kaggle.com/azzion/svm-for-beginners-tutorial

- StratifiedKFold
StratifiedKFold takes the cross validation one step further. The class distribution in the dataset is preserved in the training and test splits. Therefore, in classifications tasks with imbalanced class distributions, we should prefer StratifiedKFold over KFold.
https://towardsdatascience.com/how-to-train-test-split-kfold-vs-stratifiedkfold-281767b93869

In [7]:
import re
import pandas as pd
from os import listdir
from os.path import isfile, join

from xml.dom.minidom import parse
import xml.dom.minidom

In [10]:
"""
This is the Intro to NLP course project -- replicate the algorithm in Snow paper 
https://www.runoob.com/python3/python3-xml-processing.html
OntoNotes paper:
http://clear.colorado.edu/compsem/index.php?page=lexicalresources&sub=ontonotes
"""
import re
import pandas as pd
from os import listdir
from os.path import isfile, join

from xml.dom.minidom import parse
import xml.dom.minidom

#WordNet 2.1
import nltk
from nltk.corpus import WordNetCorpusReader
wn2 = WordNetCorpusReader("/Users/gary/Documents/perl/package/WordNet-2.1/dict", nltk.data.find("/Users/gary/Documents/perl/package/WordNet-2.1/dict"))

wn16 = WordNetCorpusReader("/Users/gary/Documents/NLP/WordNetVersions/wordnet-1.6/dict", nltk.data.find("/Users/gary/Documents/NLP/WordNetVersions/wordnet-1.6/dict"))

print(wn16.get_version())


1.6


In [114]:
print(wn16.get_version())

1.6


In [6]:
def read_xml(path, df_ontonotes,f):
    DOMTree = xml.dom.minidom.parse(path)
    collection = DOMTree.documentElement


    if collection.hasAttribute("lemma"):
        wordpos = collection.getAttribute("lemma")
        #print("Root element : %s" % collection.getAttribute("lemma"))
        #if wordpos is different with filename,use filename
        if f.replace('.xml','')!=wordpos:
            #print('f:',f,'wordpos:',wordpos)
            wordpos=f.replace('.xml','')


    # get all sense
    senses = collection.getElementsByTagName("sense")

    # get the detai info for each sense
    for sense in senses:
        groupsense=''
        group=''
        version=''
        sensenums=''
        if sense.hasAttribute("n"):
            groupsense=sense.getAttribute("n")
            #print ("n: %s" % sense.getAttribute("n"))
        if sense.hasAttribute("group"):
            group = sense.getAttribute("group")
            #print ("group: %s" % sense.getAttribute("group"))
        mapping=sense.getElementsByTagName('mappings')
        for mp in mapping:
            wn=mp.getElementsByTagName('wn')
            try:
                sensenums=wn[0].childNodes[0].data.replace('\n\t','').strip()
            except:
                sensenums=''
            for wnn in wn:
                if wnn.hasAttribute("version"):
                    version=wnn.getAttribute("version")
                    #print('version:',version)


        df_ontonotes = df_ontonotes.append({'Version':version,'WordPos':wordpos,'SenseNums':sensenums,'Group':group,'GroupSense':groupsense},
                                          ignore_index=True)
    return df_ontonotes



def generate_sensepair(df_data,df_pairs):
    """
    Purpos: generate sense pairs
    Auguments:
    df_data: input data
    df_pairs: output data
    """
    #
    wordpos_list=list(set(df_data['WordPos'].to_list()))

    for w in wordpos_list:
        df_word = df_data[df_data['WordPos']==w]
        for i in range(len(df_word)):
            pos =w.split('-')[1]
            wp_i=df_word.loc[df_word.index[i],'WordPos']
            sn_i=df_word.loc[df_word.index[i],'SenseNums']
            gp_i=df_word.loc[df_word.index[i],'GroupSense']
            sense1=wp_i.replace('-','#')+'#'+sn_i

            for j in range(i+1,len(df_word)):
                wp_j=df_word.loc[df_word.index[j],'WordPos']
                sn_j=df_word.loc[df_word.index[j],'SenseNums']
                gp_j=df_word.loc[df_word.index[j],'GroupSense']
                sense2 = wp_j.replace('-','#')+'#'+sn_j

                if gp_i==gp_j:

                    df_pairs=df_pairs.append({'Pos':pos,'Sense1':sense1,'Sense2':sense2,'Merge':'merged'},
                                                      ignore_index=True)
                else:

                    df_pairs=df_pairs.append({'Pos':pos,'Sense1':sense1,'Sense2':sense2,'Merge':'not-merged'},
                                                      ignore_index=True)
    return df_pairs


    


filepath = '/Users/gary/Documents/2020Fall/IntroNLP/project/DataSet/OntoNotes/Ontonotes-sense-groups_wn21/ontonotes-sense-groups/'

filelist = [f for f in listdir(filepath) if isfile(join(filepath, f))] #get all files' name
files_num = len(filelist)
print(files_num)
#print(filelist)
countn=0
countv=1
filesn=[]
filesv=[]
 
df_ontonotes=pd.DataFrame(columns=['Version','WordPos','SenseNums','Group','GroupSense'])

for f in filelist:
    df_ontonotes = read_xml(filepath+f,df_ontonotes,f)
    if '-n' in f:
        countn+=1
        filesn.append(f)
    if '-v' in f:
        countv+=1
        filesv.append(f)
print('n:',countn, 'v:',countv)



df_ontonotes21=df_ontonotes[df_ontonotes['Version']=='2.1']
wordpos_list=list(set(df_ontonotes21['WordPos'].to_list()))
#add pos with duplicate sense number into posduplist
posduplist=[]
for wp in wordpos_list:
    wnsensesnum=','.join(df_ontonotes21[df_ontonotes21['WordPos']==wp]['SenseNums'].to_list())

    wnsensesnumlist=wnsensesnum.split(',')
    #get the same SenseNum in different group
    if len(wnsensesnumlist)>len(set(wnsensesnumlist)):
        posduplist.append(wp)
        #print(wp,' has duplicate sense')

df_ontonotes21.to_csv('OntoNotes_Senses_raw.csv')
#one sense num in one group
df_noteclean21=df_ontonotes21[df_ontonotes21['WordPos'].isin(list(set(wordpos_list)-set(posduplist)))]  


#remove noisy data (bank-n 2&&3; drug-n 1&&2)
#df_note_clean = df_ontonotes[~(df_ontonotes['SenseNum'].str.len()>3)]

df_noteclean21= df_noteclean21[df_noteclean21['SenseNums'].notna()]
df_noteclean21['SenseNums'].replace('wn 1','1',inplace=True)
#remove row with character
df_noteclean21 = df_noteclean21[~(df_noteclean21['SenseNums'].str.contains('[A-Za-z]|&',regex=True))]

df_tmp1=df_noteclean21[df_noteclean21['SenseNums'].str.len()<3]
#df_tmp2 store cluster senses
df_tmp2 = df_noteclean21[~(df_noteclean21['SenseNums'].str.len()<3)]


#split clusters, df_tmp1 stores all the senese and  (GroupSense)
#df_tmp1.drop(columns='Unnamed: 0',inplace=True)
for i in range(len(df_tmp2)):
    Version = df_tmp2.loc[df_tmp2.index[i],'Version']
    WordPos = df_tmp2.loc[df_tmp2.index[i],'WordPos']
    SenseNums = df_tmp2.loc[df_tmp2.index[i],'SenseNums']
    Group = df_tmp2.loc[df_tmp2.index[i],'Group']
    GroupSense = df_tmp2.loc[df_tmp2.index[i],'GroupSense']
    if (',' in SenseNums):
        senses=SenseNums.split(',')
        for s in senses:
            df_tmp1=df_tmp1.append({'Version':Version,'WordPos':WordPos,'SenseNums':s,
                                   'Group':Group,'GroupSense':GroupSense},ignore_index=True)
    elif (' ' in SenseNums):
            senses=SenseNums.split(' ')
            for s in senses:
                df_tmp1=df_tmp1.append({'Version':Version,'WordPos':WordPos,'SenseNums':s,
                                       'Group':Group,'GroupSense':GroupSense},ignore_index=True)
    elif ('.' in SenseNums):
            senses=SenseNums.split('.')
            for s in senses:
                df_tmp1=df_tmp1.append({'Version':Version,'WordPos':WordPos,'SenseNums':s,
                                       'Group':Group,'GroupSense':GroupSense},ignore_index=True)

df_tmp1.to_csv('OntoNotes_Senses_cleaned.csv')


df_pairs=pd.DataFrame(columns=['Pos','Sense1','Sense2','Merge'])

#generate pairs data
df_pairs=generate_sensepair(df_tmp1,df_pairs)
df_pairs.to_csv('OntoNotes_SensesPairs.csv')



1116
n: 460 v: 657


In [178]:

file_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
df_pairs= pd.read_csv(file_path+'OntoNotes_SensesPairs.csv')
#remove wrong senses (no sense number)
df_pairs = df_pairs[df_pairs['Sense1'].str.contains('[0-9]')]
df_pairs = df_pairs[df_pairs['Sense2'].str.contains('[0-9]')]

df_pairs.to_csv('OntoNotes_SensesPairs.csv',index=False)




In [190]:
file_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
df_pairs= pd.read_csv(file_path+'OntoNotes_SensesPairs.csv')
df_pairs['Sense2']=df_pairs['Sense2'].str.replace('team#n#2-','team#n#2')

df_pairs[df_pairs['Sense2']=='team#n#2-']
df_pairs.to_csv('OntoNotes_SensesPairs.csv',index=False)


In [194]:
#use perl to get the following features: antonyms, deriv, pertainyms, verbgroups
file_path='/Users/gary/Documents/2020Fall/IntroNLP/project/note_pairs_wncourpus_similarity.csv'
df_perl=pd.read_csv(file_path)
df_perl




,sense1,ants1,deri1,vgrp1,pert1,sense2,ants2,deri2,vgrp2,pert2,merge
0,Sense1,NaN,NaN,NaN,NaN,Sense2,NaN,NaN,NaN,NaN,Merge
1,consult#v#3,NaN,consultation#n#2; consultation#n#1,NaN,NaN,consult#v#4,NaN,consultive#a#1; consultative#a#1; consultatory...,NaN,NaN,not-merged
2,consult#v#3,NaN,consultation#n#2; consultation#n#1,NaN,NaN,consult#v#1,NaN,consultive#a#1; consultative#a#1; consultatory...,NaN,NaN,not-merged
3,consult#v#3,NaN,consultation#n#2; consultation#n#1,NaN,NaN,consult#v#2,NaN,consultation#n#2; consultation#n#3,NaN,NaN,not-merged
4,consult#v#4,NaN,consultive#a#1; consultative#a#1; consultatory...,NaN,NaN,consult#v#1,NaN,consultive#a#1; consultative#a#1; consultatory...,NaN,NaN,not-merged
...,...,...,...,...,...,...,...,...,...,...,...
12332,discover#v#4,NaN,discovery#n#2; discovery#n#3; discovery#n#1,NaN,NaN,discover#v#8,NaN,NaN,NaN,NaN,merged
12333,discover#v#5,NaN,discovery#n#2; discovery#n#1,NaN,NaN,discover#v#7,NaN,discovery#n#2; discovery#n#3; discovery#n#1,NaN,NaN,merged
12334,discover#v#5,NaN,discovery#n#2; discovery#n#1,NaN,NaN,discover#v#8,NaN,NaN,NaN,NaN,merged
12335,discover#v#7,NaN,discovery#n#2; discovery#n#3; discovery#n#1,NaN,NaN,discover#v#8,NaN,NaN,NaN,NaN,merged


In [207]:
df_perl[df_perl[' vgrp1']==df_perl[' vgrp2']]

,sense1,ants1,deri1,vgrp1,pert1,sense2,ants2,deri2,vgrp2,pert2,merge
244,feel#v#10,NaN,feel#n#3,feel#v#8,NaN,feel#v#13,NaN,feel#n#4,feel#v#8,NaN,merged
1972,convert#v#6,NaN,conversion#n#3,convert#v#8,NaN,convert#v#7,NaN,NaN,convert#v#8,NaN,merged
2953,kill#v#1,NaN,killable#a#1; kill#n#1; killer#n#2; killer#n#1...,kill#v#3,NaN,kill#v#10,NaN,kill#n#1,kill#v#3,NaN,merged
4859,play#v#1,NaN,play#n#9; play#n#3; player#n#4; player#n#1; pl...,take_on#v#5,NaN,play#v#31,NaN,play#n#4,take_on#v#5,NaN,merged
4939,play#v#32,NaN,play#n#3,take_on#v#5; play#v#15,NaN,play#v#33,NaN,play#n#3,take_on#v#5; play#v#15,NaN,merged
5187,play#v#14,NaN,play#n#2; play#n#1,playact#v#1,NaN,play#v#26,NaN,play#n#1; play#n#2,playact#v#1,NaN,merged
5220,play#v#10,NaN,play#n#16,play#v#29,NaN,play#v#30,NaN,play#n#16,play#v#29,NaN,merged
5773,fail#v#6,pass#v#16,failing#n#2,flush_it#v#1,NaN,fail#v#8,NaN,failure#n#2; failing#n#1,flush_it#v#1,NaN,merged
7520,read#v#10,NaN,NaN,read#v#6,NaN,read#v#5,NaN,NaN,read#v#6,NaN,not-merged
7819,tap#v#10,NaN,tap#n#6; tapper#n#1,tap#v#6,NaN,tap#v#11,NaN,tap#n#6,tap#v#6,NaN,merged


In [201]:
df_features[df_features['deriv']>0]

,Unnamed: 0,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,...,antonyms,deriv,lemmas,verbgroup,verbframe,hyper_min,hyper_max,topic_similarity,wn_domain_feature,wn_oed_feature
0,0,v,consult#v#3,consult#v#4,not-merged,1.945910,4.0,0.000000,1.0,0.000000,...,0.0,1.0,1.0,0.0,1.0,0.5,5.5,NaN,1.0,1.0
1,1,v,consult#v#3,consult#v#1,not-merged,1.386294,0.0,0.073668,5.0,0.301704,...,0.0,1.0,1.0,0.0,1.0,2.0,4.0,NaN,1.0,1.0
2,2,v,consult#v#3,consult#v#2,not-merged,0.847298,0.0,0.051132,7.0,0.000000,...,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0
3,3,v,consult#v#4,consult#v#1,not-merged,1.252763,0.0,0.000000,2.0,0.000000,...,0.0,1.0,1.0,0.0,1.0,2.5,4.5,NaN,1.0,1.0
4,4,v,consult#v#4,consult#v#2,not-merged,0.767255,0.0,0.000000,3.0,0.000000,...,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12899,12920,v,discover#v#3,discover#v#5,merged,1.722767,0.0,0.067229,5.0,0.000000,...,0.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,1.0,1.0
12900,12921,v,discover#v#3,discover#v#7,merged,2.233592,0.0,0.059452,12.0,0.000000,...,0.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,1.0,1.0
12902,12923,v,discover#v#4,discover#v#5,merged,1.540445,0.0,0.060483,8.0,0.000000,...,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0
12903,12924,v,discover#v#4,discover#v#7,merged,1.945910,0.0,0.054114,13.0,0.000000,...,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0


In [64]:
'''
Build feature space:
1, twin: the number of shared synonyms between two synsets
2, antonym: whether two synsets share an antonym
3, pertainym: whether two synsets share an pertainym
4, deriv: whether two synsets share derivationally related forms
5, verbgrp: whether two verb synsets are linked in a VERBGROUP (indicating semantic similarity)
6, verbfrm: whether two verb synsets share a VERBFRAM (indicating syntactic similarity) 

'''
#wordnet 2.1
df_pairs=pd.read_csv('OntoNotes_SensesPairs.csv')
#df_pairs
for i in range(len(df_pairs)):
    pertainyms1=[]
    antonyms1=[]
    deriv1=[]
    lemmas1=[]
    verbgroup1=[]
    verbframe1=[]
    pertainyms2=[]
    antonyms2=[]
    deriv2=[]
    lemmas2=[]
    verbgroup2=[]
    verbframe2=[]
    pertainymsflag=''
    antonymsflag=''
    derivflag=''
    lemmasflag=''
    verbgroupflag=''
    verbframeflag=''
    hyper_min=''
    hyper_max=''

    sense1 = df_pairs.loc[df_pairs.index[i],'Sense1']
    sense1 = sense1.replace('#','.')
    sense2 = df_pairs.loc[df_pairs.index[i],'Sense2']
    sense2 = sense2.replace('#','.')
    pos = df_pairs.loc[df_pairs.index[i],'Pos']
    
    # sense1's twin, pertainyms, antonyms, derivationally_related_forms,
    # verb group, frame id, hyper_min, hyper_max
    try:
        
        #WN, WNMax feature
        #calculate hyper distance
        sense1_hyper = wn2.synset(sense1)
        sense1_hypers = lambda s: s.hypernyms()
        hyper1= list(sense1_hyper.closure(sense1_hypers))
        
        sense2_hyper = wn2.synset(sense2)
        sense2_hypers = lambda s: s.hypernyms()
        hyper2 = list(sense2_hyper.closure(sense2_hypers))
           
        #find the nearest hyper, and average the distance as the least distance
        for h1 in hyper1:
            if h1 in hyper2:
                hyper_min = (hyper1.index(h1)+hyper2.index(h1))/2
                break
        


        #find the farest hyper, and average the distance as the largest distance
        for h1 in reversed(hyper1):
            if h1 in hyper2:
                hyper_max = (hyper1.index(h1)+hyper2.index(h1))/2
                break
                
        
        for lemma in wn2.synset(sense1).lemmas():
            
            pertainyms1=pertainyms1+lemma.pertainyms()
            antonyms1=antonyms1+lemma.antonyms()
            deriv1=deriv1+lemma.derivationally_related_forms()
            lemmas1.append(lemma.name())
            
            if pos=='v':
                verbgroup1=verbgroup1+lemma.verb_groups()
                verbframe1=verbframe1+lemma.frame_ids()

                
        # sense2's pertainyms, antonyms, derivationally_related_forms
        for lemma in wn2.synset(sense2).lemmas():
            
            pertainyms2=pertainyms2+lemma.pertainyms()
            antonyms2=antonyms2+lemma.antonyms()
            deriv2=deriv2+lemma.derivationally_related_forms()
            lemmas2.append(lemma.name())
             
            #verb group, verb frame
            if pos=='v':
                verbgroup2=verbgroup2+lemma.verb_groups()
                verbframe2=verbframe2+lemma.frame_ids()
                
      
        if (len(set(pertainyms1)&set(pertainyms2))>0):
            pertainymsflag=1
        else:
            pertainymsflag=0

        if (len(set(antonyms1)&set(antonyms2))>0):
            antonymsflag=1
        else:
            antonymsflag=0 

        if (len(set(deriv1)&set(deriv2))>0):
            derivflag=1
        else:
            derivflag=0 
        
        #verb group, verb frame
        if pos=='v':
            if (len(set(verbgroup1)&set(verbgroup2))>0):
                verbgroupflag=1
            else:
                verbgroupflag=0
                
            if (len(set(verbframe1)&set(verbframe2))>0):
                verbframeflag=1
            else:
                verbframeflag=0
            

        
        lemmasflag=len(set(lemmas1)&set(lemmas2))
       
        
        df_pairs.loc[df_pairs.index[i],'pertainyms']=pertainymsflag
        df_pairs.loc[df_pairs.index[i],'antonyms']=antonymsflag
        df_pairs.loc[df_pairs.index[i],'deriv']=derivflag
        df_pairs.loc[df_pairs.index[i],'lemmas']=lemmasflag
        
        df_pairs.loc[df_pairs.index[i],'verbgroup']=verbgroupflag
        df_pairs.loc[df_pairs.index[i],'verbframe']=verbframeflag
         
        df_pairs.loc[df_pairs.index[i],'hyper_min']=hyper_min
        df_pairs.loc[df_pairs.index[i],'hyper_max']=hyper_max

        
    except:
        None
    
df_pairs.to_csv('OntoNotes_SensesPairs_WNFeatures.csv')    
    
    # 
    
        


In [19]:
#map WN2.1 to WN1.6 for topic signuature
#this mapping only for nouns is based on sensekey, headword, and pos
# the verbs are also included
'''
The basic idea is that if two synsets from different WN versions share the same ‘word’+’pos’+’sense key’, 
then they are identical. This algorithm is tested on WN2.x and WN3.0, and achieved 97% accuracy.
'''

df_pairs=pd.read_csv('OntoNotes_SensesPairs.csv')

df_wn21wn16=pd.DataFrame(columns=['pos','sense1_wn21','sense2_wn21','sense1_wn16','sense2_wn16','offset1_wn16','offset2_wn16','sense1_wn21_def','sense1_wn16_def','sense2_wn21_def','sense2_wn16_def'])

#df_pairs_n=df_pairs[df_pairs['Pos']=='n']
#total_pair=len(df_pairs_n)
total_pair=len(df_pairs)
for i in range(total_pair):
    if (i%1000 ==0):
        print('processing {}/{}'.format(i,total_pair))
    sense1_wn21 = df_pairs.loc[df_pairs.index[i],'Sense1']
    headword=sense1_wn21.split('#')[0]
    sense1_wn21 = sense1_wn21.replace('#','.')
    sense2_wn21 = df_pairs.loc[df_pairs.index[i],'Sense2']
    sense2_wn21 = sense2_wn21.replace('#','.')
    pos = df_pairs.loc[df_pairs.index[i],'Pos']
    
    try:
        for lemma1 in wn2.synset(sense1_wn21).lemmas():
            sense1_wn21_def = wn2.synset(sense1_wn21).definition()
            #get sense key for wn21 sense1
            wn2_sense1_key=lemma1.key()
            sense1_wn16=''
            #enumerate all wn synsests for headword#pos,find the same sense key
            for synset1 in wn16.synsets(headword, pos):
                lemmas = synset1.lemmas()
                for l1 in lemmas:
                    #if sense key equal, keep the wn16 synset name
                    if l1.key() == wn2_sense1_key:
                        sense1_wn16=synset1.name()
                        sense1_wn16_def= synset1.definition()
                        offset1_wn16 = str(synset1.offset()).zfill(8) +'-'+ synset1.pos()


        for lemma2 in wn2.synset(sense2_wn21).lemmas():
            sense2_wn21_def = wn2.synset(sense2_wn21).definition()
            wn2_sense2_key=lemma2.key()
            sense2_wn16=''
            for synset2 in wn16.synsets(headword, pos):
                lemmas = synset2.lemmas()
                for l2 in lemmas:
                    if l2.key() == wn2_sense2_key:
                        sense2_wn16=synset2.name()
                        sense2_wn16_def= synset2.definition()
                        offset2_wn16 = str(synset2.offset()).zfill(8) +'-'+ synset2.pos()
    
        df_wn21wn16=df_wn21wn16.append({'pos':pos,'sense1_wn21':sense1_wn21,'sense2_wn21':sense2_wn21,'sense1_wn16':sense1_wn16.replace('.0','.'),'sense2_wn16':sense2_wn16.replace('.0','.'),
                                        'offset1_wn16':offset1_wn16,'offset2_wn16':offset2_wn16,'sense1_wn21_def':sense1_wn21_def,'sense1_wn16_def':sense1_wn16_def,'sense2_wn21_def':sense2_wn21_def,'sense2_wn16_def':sense2_wn16_def},
                                   ignore_index=True)
    except:
        None
    
df_wn21wn16.to_csv('WN21mapWn16_full.csv',index=False)


processing 0/12914
processing 1000/12914
processing 2000/12914
processing 3000/12914
processing 4000/12914
processing 5000/12914
processing 6000/12914
processing 7000/12914
processing 8000/12914
processing 9000/12914
processing 10000/12914
processing 11000/12914
processing 12000/12914


In [18]:
df_pairs

,Unnamed: 0,Pos,Sense1,Sense2,Merge
0,0,v,consult#v#3,consult#v#4,not-merged
1,1,v,consult#v#3,consult#v#1,not-merged
2,2,v,consult#v#3,consult#v#2,not-merged
3,3,v,consult#v#4,consult#v#1,not-merged
4,4,v,consult#v#4,consult#v#2,not-merged
...,...,...,...,...,...
12909,12909,v,discover#v#5,discover#v#8,merged
12910,12910,v,discover#v#7,discover#v#8,merged
12911,12911,n,restriction#n#1,restriction#n#,not-merged
12912,12912,n,restriction#n#1,restriction#n#3,not-merged


In [8]:
'''
Topic signature data:
Each file is named according to the following pattern:
measure.BNCfilt.target_word.PoS.sense.txt.bz2
for instance: tf_idf.BNCfilt.church.n.1.txt.bz2, which corresponds to the
topic signature of the first sense of church built using the tf.idf
measure.
- Each file is compresed with bzip2 (http://sources.redhat.com/bzip2/)

1) read df_wn21wn16 and copy the corresponding files into another folder
2) unzip these files
3) calculate the cosine similarity of each pair 
'''
import os
import os.path

df_wn21wn16=pd.read_csv('WN21mapWn16.csv')
directory='/Users/gary/Documents/2020Fall/IntroNLP/project/FeatureSpace/TopicSignatures/signatures_lem/'
des_dir='/Users/gary/Documents/2020Fall/IntroNLP/project/FeatureSpace/TopicSingatures_SnowProject/'

total_pair = len(df_wn21wn16)
senses=[]
for i in range(total_pair):
    sense1=df_wn21wn16.loc[df_wn21wn16.index[i],'sense1_wn16']
    sense2=df_wn21wn16.loc[df_wn21wn16.index[i],'sense2_wn16']
    senses.append(sense1)
    senses.append(sense2)

for s in senses:
    if s!=s:
        senses.remove(s)
#copy corresponding files to des_dir
j=0
for s in set(senses):
    try:
        if (j%500==0):
            print('processing {}/{}'.format(j,len(set(senses))))
        for dirpath, dirnames, filenames in os.walk(directory):
            for filename in [f for f in filenames if "BNC_filt."+s in f]:
                cp='cp '+os.path.join(dirpath, filename)+' '+des_dir+filename
                #copy files 
                os.system(cp)
                #unzip='bzip2 -d '+
                #print(filename)
        j+=1
    except:
        None


processing 0/933
processing 0/933
processing 0/933
processing 0/933
processing 0/933
processing 0/933
processing 0/933
processing 0/933
processing 0/933
processing 0/933
processing 500/933
processing 500/933
processing 500/933
processing 500/933
processing 500/933


In [35]:
#calculate cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
a=[[1,3,2],[2,2,1]]
b=np.array([[1,3,2]])
c=np.array([[2,2,1]])
cosine_similarity(b,c) #0.89087081
b.shape

(1, 3)

In [59]:
#build topic signature feature for POS=n
#calculate cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

df_wn21wn16=pd.read_csv('WN21mapWn16.csv')


des_dir='/Users/gary/Documents/2020Fall/IntroNLP/project/FeatureSpace/TopicSingatures_SnowProject/'
filelist = [f for f in listdir(des_dir) if isfile(join(des_dir, f))] #get all files' name

#add corresponding filename to the dataframe
for i in range(len(df_wn21wn16)):
    if i%200==0:
        print('processing {}/{}'.format(i,len(df_wn21wn16)))
    sense1= df_wn21wn16.loc[df_wn21wn16.index[i],'sense1_wn16']
    sense2= df_wn21wn16.loc[df_wn21wn16.index[i],'sense2_wn16']
    for f in filelist:
        if '.'+str(sense1)+'.' in f:
            df_wn21wn16.loc[df_wn21wn16.index[i],'sense1_topic']=f
        if '.'+str(sense2)+'.' in f:
            df_wn21wn16.loc[df_wn21wn16.index[i],'sense2_topic']=f


#calculate the similarity
df_topic=df_wn21wn16[(df_wn21wn16['sense1_topic'].notna())&(df_wn21wn16['sense2_topic'].notna())]

for j in range(len(df_topic)):
    topic1 = df_topic.loc[df_topic.index[j],'sense1_topic']
    topic2 = df_topic.loc[df_topic.index[j],'sense2_topic']
    
    df_tp1=pd.read_csv(des_dir+topic1,header=None,sep=' ')
    df_tp1.columns=['word','tfidf','num']
    tp1=df_tp1['tfidf'].to_list()
    
    df_tp2=pd.read_csv(des_dir+topic2,header=None,sep=' ')
    df_tp2.columns=['word','tfidf','num']
    tp2=df_tp2['tfidf'].to_list()
    
    if len(tp1)>len(tp2):
        tp1 = tp1[:len(tp2)]
    else:
        tp2 = tp2[:len(tp1)]
        
    tp1 = np.array(tp1).reshape(1,len(tp1))
    tp2 = np.array(tp2).reshape(1,len(tp2))
    
    df_topic.loc[df_topic.index[j],'topic_similarity']=cosine_similarity(tp1,tp2)[0,0] 
    

df_topic.to_csv('WN21mapWn16_topic_similarity.csv',index=False)    


processing 0/2534
processing 200/2534
processing 400/2534
processing 600/2534
processing 800/2534
processing 1000/2534
processing 1200/2534
processing 1400/2534
processing 1600/2534
processing 1800/2534
processing 2000/2534
processing 2200/2534
processing 2400/2534


/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

...

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [60]:
df_topic

,pos,sense1_wn21,sense2_wn21,sense1_wn16,sense2_wn16,sense1_wn21_def,sense1_wn16_def,sense2_wn21_def,sense2_wn16_def,sense1_topic,sense2_topic,topic_similarity
0,n,chain.n.1,chain.n.2,chain.n.1,chain.n.2,a series of things depending on each other as ...,a series of things depending on each other as ...,(chemistry) a series of linked atoms (generall...,a series of linked atoms (generally in an orga...,tf_idfBNC_filt.chain.n.1.txt,tf_idfBNC_filt.chain.n.2.txt,0.725049
1,n,chain.n.1,chain.n.4,chain.n.1,chain.n.4,a series of things depending on each other as ...,a series of things depending on each other as ...,a number of similar establishments (stores or ...,a number of similar establishments (stores or ...,tf_idfBNC_filt.chain.n.1.txt,tf_idfBNC_filt.chain.n.4.txt,0.787393
2,n,chain.n.1,chain.n.9,chain.n.1,chain.n.9,a series of things depending on each other as ...,a series of things depending on each other as ...,a necklace made by a stringing objects together,a necklace made by a stringing objects togethe...,tf_idfBNC_filt.chain.n.1.txt,tf_idfBNC_filt.chain.n.9.txt,0.993389
3,n,chain.n.1,chain.n.8,chain.n.1,range.n.5,a series of things depending on each other as ...,a series of things depending on each other as ...,a series of hills or mountains,a series of hills or mountains,tf_idfBNC_filt.chain.n.1.txt,tf_idfBNC_filt.range.n.5.txt,0.691872
4,n,chain.n.1,chain.n.6,chain.n.1,chain.n.6,a series of things depending on each other as ...,a series of things depending on each other as ...,a unit of length,a unit of length,tf_idfBNC_filt.chain.n.1.txt,tf_idfBNC_filt.chain.n.6.txt,0.814840
...,...,...,...,...,...,...,...,...,...,...,...,...
2517,n,deposit.n.1,deposit.n.7,deposit.n.1,deposit.n.6,the phenomenon of sediment or gravel accumulating,the phenomenon of sediment or gravel accumulating,a payment given as a guarantee that an obligat...,a payment given as a guarantee that an obligat...,tf_idfBNC_filt.deposit.n.1.txt,tf_idfBNC_filt.deposit.n.6.txt,0.941813
2530,n,deposit.n.9,deposit.n.6,deposit.n.8,deposit.n.5,the act of putting something somewhere,the act of depositing,money given as security for an article acquire...,money given as security for an article acquire...,tf_idfBNC_filt.deposit.n.8.txt,tf_idfBNC_filt.deposit.n.5.txt,0.733986
2531,n,deposit.n.9,deposit.n.7,deposit.n.8,deposit.n.6,the act of putting something somewhere,the act of depositing,a payment given as a guarantee that an obligat...,a payment given as a guarantee that an obligat...,tf_idfBNC_filt.deposit.n.8.txt,tf_idfBNC_filt.deposit.n.6.txt,0.845284
2532,n,deposit.n.6,deposit.n.7,deposit.n.5,deposit.n.6,money given as security for an article acquire...,money given as security for an article acquire...,a payment given as a guarantee that an obligat...,a payment given as a guarantee that an obligat...,tf_idfBNC_filt.deposit.n.5.txt,tf_idfBNC_filt.deposit.n.6.txt,0.834040


In [32]:
#create feature based on WN Domain dataset
#WordNet Domain dataset; two datafile, one for WN1.6, one for WN2.0
#Snow didn't mention which one they use, but I use WN1.6, since I mapped 2.1 to 1.6
#Snow created two features, but I only created the first one, since I was not sure about
#the second one.
#- "wn-domains-2.0-20050210" contains the mapping between Princeton WordNet 1.6 synsets and their corresponding domains. The format is as above.

df_wn21wn16=pd.read_csv('WN21mapWn16_full.csv')
df_wn_domain=pd.read_csv('/Users/gary/Documents/2020Fall/IntroNLP/project/FeatureSpace/wn-domains-3.2/wn-domains-2.0-20050210',
                         sep='\t',header=None)


df_wn_domain.columns=['offset-pos','domain']
#filter offers that not in domain data
#wn16_offsets_lst = list(set(df_wn21wn16['offset1_wn16'].to_list()+df_wn21wn16['offset2_wn16'].to_list()))

#domain_offsets_lst=list(set(df_wn_domain['offset-pos'].to_list()))

#overlap_offsets_lst = list(set(wn16_offsets_lst)&set(domain_offsets_lst))

for i in range(len(df_wn21wn16)):
    offset1=df_wn21wn16.loc[df_wn21wn16.index[i],'offset1_wn16']
    offset2=df_wn21wn16.loc[df_wn21wn16.index[i],'offset2_wn16']
    dm1=df_wn_domain[df_wn_domain['offset-pos']==offset1]['domain'].to_list()
    dm2=df_wn_domain[df_wn_domain['offset-pos']==offset2]['domain'].to_list()
    if (len(dm1)==1 and len(dm2)==1):
        if dm1[0]==dm2[0]:
            feature=1
    else:
        feature=0
    
    df_wn21wn16.loc[df_wn21wn16.index[i],'wn_domain_feature']=feature

df_wn21wn16.to_csv('WN21mapWn16_WN_Domain_feature.csv',index=False)


In [226]:
df_wn_domain

,offset-pos,domain
0,00001740-n,factotum
1,00002086-n,biology
2,00002880-n,biology
3,00003011-n,biology
4,00003095-n,biology
...,...,...
99637,00509912-r,factotum
99638,00509991-r,baseball play
99639,00510095-r,factotum
99640,00510176-r,linguistics


In [67]:
#create feature based on mapping between WN and OED, the mapping is based on WN21
#read mapping file
file_map_wnoed=open('/Users/gary/Documents/2020Fall/IntroNLP/project/FeatureSpace/sense_clusters-21.senses','r')

#read sysnset pairs
df_pairs=pd.read_csv('OntoNotes_SensesPairs.csv')


#df_pairs_n=df_pairs[df_pairs['Pos']=='n']
#total_pair=len(df_pairs_n)
total_pair=len(df_pairs)
for i in range(total_pair):
    if (i%1000 ==0):
        print('processing {}/{}'.format(i,total_pair))
    sense1_wn21 = df_pairs.loc[df_pairs.index[i],'Sense1']
    headword=sense1_wn21.split('#')[0]
    sense1_wn21 = sense1_wn21.replace('#','.')
    sense2_wn21 = df_pairs.loc[df_pairs.index[i],'Sense2']
    sense2_wn21 = sense2_wn21.replace('#','.')
    pos = df_pairs.loc[df_pairs.index[i],'Pos']
    
    sense1_keys=''
    sense2_keys=''
    try:
        for lemma1 in wn2.synset(sense1_wn21).lemmas():
            #get sense key for wn21 sense1
            sense1_keys=sense1_keys+lemma1.key()+';'
        for lemma2 in wn2.synset(sense2_wn21).lemmas():
            #get sense key for wn21 sense1
            sense2_keys=sense2_keys+lemma1.key()+';'
    except:
        None

    df_pairs.loc[df_pairs.index[i],'Sense1_keys']=sense1_keys
    df_pairs.loc[df_pairs.index[i],'Sense2_keys']=sense2_keys

#filter sense keys is null
df_pairs = df_pairs[(df_pairs['Sense1_keys'].str.len()>5)&(df_pairs['Sense1_keys'].str.len()>5)]

#get sense clusterings from the mapping between WN and OED
lines = file_map_wnoed.readlines()
count=0
wnoed_lst=[] # a list of sets.
for line in lines:
    count+=1
    line=line.strip('\n')
    if ' ' in line:
        wnoed_lst.append(set(line.split(' ')))

for i in range(len(df_pairs)):
    if i%1000 ==0:
        print('processing {}/{}'.format(i,total_pair))
    sense1_key=df_pairs.loc[df_pairs.index[i],'Sense1_keys']
    sk1_set=set(sense1_key.split(';'))
    
    sense2_key=df_pairs.loc[df_pairs.index[i],'Sense2_keys']
    sk2_set=set(sense2_key.split(';'))
    
    feature=0
    for  w in wnoed_lst:
        sk1_len=len(sk1_set & w)
        sk2_len=len(sk2_set & w)
        if (sk1_len>0 and sk2_len>0):
            feature=1
            break
    
    df_pairs.loc[df_pairs.index[i],'wn_oed_feature']=feature
    
df_pairs.to_csv('WN_OED_Map_Feature_OntoNotes_SensesPairs.csv',index=False)


processing 0/12914
processing 1000/12914
processing 2000/12914
processing 3000/12914
processing 4000/12914
processing 5000/12914
processing 6000/12914
processing 7000/12914
processing 8000/12914
processing 9000/12914
processing 10000/12914
processing 11000/12914
processing 12000/12914
processing 0/12914
processing 1000/12914
processing 2000/12914
processing 3000/12914
processing 4000/12914
processing 5000/12914
processing 6000/12914
processing 7000/12914
processing 8000/12914
processing 9000/12914
processing 10000/12914
processing 11000/12914
processing 12000/12914


In [68]:
df_pairs[(df_pairs['Sense1_keys'].str.len()>5)&(df_pairs['Sense1_keys'].str.len()>5)]

,Unnamed: 0,Pos,Sense1,Sense2,Merge,Sense1_keys,Sense2_keys,wn_oed_feature
0,0,v,consult#v#3,consult#v#4,not-merged,confer%2:32:00::;confabulate%2:32:01::;confab%...,consult%2:32:05::;,1.0
1,1,v,consult#v#3,consult#v#1,not-merged,confer%2:32:00::;confabulate%2:32:01::;confab%...,consult%2:32:05::;consult%2:32:05::;,1.0
2,2,v,consult#v#3,consult#v#2,not-merged,confer%2:32:00::;confabulate%2:32:01::;confab%...,consult%2:32:05::;consult%2:32:05::;consult%2:...,1.0
3,3,v,consult#v#4,consult#v#1,not-merged,consult%2:32:00::;,consult%2:32:00::;consult%2:32:00::;,1.0
4,4,v,consult#v#4,consult#v#2,not-merged,consult%2:32:00::;,consult%2:32:00::;consult%2:32:00::;consult%2:...,1.0
...,...,...,...,...,...,...,...,...
12908,12908,v,discover#v#5,discover#v#7,merged,fall_upon%2:40:13::;strike%2:40:00::;come_upon...,discover%2:40:00::;,1.0
12909,12909,v,discover#v#5,discover#v#8,merged,fall_upon%2:40:13::;strike%2:40:00::;come_upon...,discover%2:40:00::;discover%2:40:00::;discover...,1.0
12910,12910,v,discover#v#7,discover#v#8,merged,discover%2:39:00::;,discover%2:39:00::;discover%2:39:00::;discover...,1.0
12911,12911,n,restriction#n#1,restriction#n#,not-merged,restriction%1:09:00::;limitation%1:09:00::;,,0.0


In [65]:
file_map_wnoed=open('/Users/gary/Documents/2020Fall/IntroNLP/project/FeatureSpace/sense_clusters-21.senses','r')


lines = file_map_wnoed.readlines()
count=0
wnoed_lst=[]
for line in lines:
    count+=1
    line=line.strip('\n')
    if ' ' in line:
        wnoed_lst.append(set(line.split(' ')))

print('total',count)
wnoed_lst

total 29974


[{'A%1:23:00::', 'A%1:23:01::'},
 {'A%1:08:00::', 'A%1:10:00::', 'A%1:27:00::', 'A%1:27:02::'},
 {'AA%1:10:00::', 'AA%1:14:00::', 'aa%1:27:00::'},
 {'AB%1:08:00::', 'AB%1:10:00::', 'Ab%1:28:00::', 'ab%1:08:01::'},
 {'AMP%1:27:00::', 'amp%1:23:00::'},
 {'Aborigine%1:18:01::', 'aborigine%1:18:00::'},
 {'Acre%1:15:00::', 'Acre%1:15:01::', 'acre%1:23:00::'},
 {'Advent%1:11:00::', 'Advent%1:28:01::'},
 {'Aeolian%3:01:00::', 'aeolian%3:01:02::'},
 {'Agriculture%1:14:01::',
  'agriculture%1:04:00::',
  'agriculture%1:04:01::',
  'agriculture%1:14:00::'},
 {'Alpine%3:01:00::', 'alpine%3:01:01::', 'alpine%5:00:00:upland:00'},
 {'American%1:18:00::', 'American%1:18:01::'},
 {'American%3:01:00::', 'American%3:01:01::'},
 {'Amphisbaena%1:05:00::', 'amphisbaena%1:18:00::'},
 {'Ana%1:18:00::', 'ana%1:14:00::'},
 {'Angelus%1:10:00::', 'angelus%1:11:00::'},
 {'Angora%1:05:01::',
  'Angora%1:05:02::',
  'Angora%1:05:03::',
  'Angora%1:15:00::'},
 {'Annunciation%1:10:00::', 'Annunciation%1:28:00::'},
 {

In [64]:
a='abc'
a.split(' ')

['abc']

In [ ]:
#look up the "POS, Sense Keys, HeadWord" 

#algorithm that mapping the 5K list to WordNet 3.0 by "POS+SenseKeys+HeadWord"
import nltk
import re
from nltk.corpus import wordnet as wn
import pandas as pd
#mapping the 5K list to current WordNet by sense keys
#there is no mapping based on gloss, the gloss of each synset in 5K are only few words
f = open("5K.clean.txt")
line = f.readline()
out_list = []
i=0

mapping_result = pd.DataFrame(columns=['POS','SenseKeys', 'HeadWord','Gloss_5K','Gloss_WordNet','Synset','SynsetOffset'])

while line:
    flag = True
    #print(line,line[0],'sense key:',re.findall(r'[[](.*?)[]]', line)[0], 'word:',re.findall(r'[[](.*?)[]]', line)[1] )
    pos = line[0]
    word = re.findall(r'[[](.*?)[]]', line)[1]
    sense_key = re.findall(r'[[](.*?)[]]', line)[0]
    position = line.find(']',line.find(']')+2)  #position of second ']',s.find(']') return the lowest index
    if position+4<len(line): #there is gloss
        gloss = line[position+2:] #
    else:
        gloss = ""
    inner_list = []
    for synset in wn.synsets(word, pos):
        lemmas = synset.lemmas()
        
        for l in lemmas:
            if l.key() == sense_key:
                gloss_wn = synset.definition()
                offset = "wn:"+str(synset.offset()).zfill(8)+ synset.pos()
                #i+=1
                #print(i)
                flag = False
                inner_list.append(synset)
                inner_list.append(sense_key)
                inner_list.append(word)
                inner_list.append(pos)
                out_list.append(inner_list)
                mapping_result = mapping_result.append({'POS':pos, 'SenseKeys':sense_key,'HeadWord':word,
                                      'Gloss_5K':gloss, 'Gloss_WordNet': gloss_wn, 'Synset': synset,'SynsetOffset':offset},ignore_index=True)
    #if flag:
        #print(line)
    line = f.readline()
    
f.close()
mapping_result.to_csv('Mapping_result_5K_WN.csv',sep=',')

In [88]:
sense1_hyper = wn2.synset("car.n.01")
sense1_hypers = lambda s: s.hypernyms()
hyper1= list(sense1_hyper.closure(sense1_hypers))

sense2_hyper = wn2.synset("boat.n.01")
sense2_hypers = lambda s: s.hypernyms()
hyper2 = list(sense2_hyper.closure(sense2_hypers))

#find the nearest hyper, and average the distance as the least distance
for h1 in hyper1:
    if h1 in hyper2:
        hyper_min = (hyper1.index(h1)+hyper2.index(h1))/2
        break



#find the farest hyper, and average the distance as the largest distance
for h1 in reversed(hyper1):
    if h1 in hyper2:
        hyper_max = (hyper1.index(h1)+hyper2.index(h1))/2
        break
print(hyper_min, hyper_max)

3.0 10.0


In [105]:
len(set(df_pairs['verbgroup'].to_list()))
df_pairs[df_pairs['verbgroup'].notna()]['verbgroup'].to_list()
set(df_pairs[df_pairs['hyper_min'].notna()]['hyper_min'].to_list())
38*6+5

233

In [86]:
#car = wn2.synset('car.n.1')
#print(car.hypernyms(),car.hypernyms()[0].hypernyms())
panda = wn2.synset("car.n.01")
hyper = lambda s: s.hypernyms()
hyper1= list(panda.closure(hyper))

panda = wn2.synset("boat.n.01")
hyper = lambda s: s.hypernyms()
hyper2 = list(panda.closure(hyper))

#find the nearest hyper, and average the distance as the least distance
for h1 in hyper1:
    if h1 in hyper2:
        print(h1)
        print((hyper1.index(h1)+hyper2.index(h1))/2)
        break
#find the farest hyper, and average the distance as the largest distance
for h1 in reversed(hyper1):
    if h1 in hyper2:
        print(h1)
        print((hyper1.index(h1)+hyper2.index(h1))/2)
        break

#for lemma in wn2.lemmas('eat', 'v'):
    #print(lemma.verb_groups())

Synset('vehicle.n.01')
3.0
Synset('entity.n.01')
10.0


In [49]:
pertainyms1=[]
antonyms1=[]
deriv1=[]
pertainyms2=[]
antonyms2=[]
deriv=[]
for lemma in wn2.synset('good.n.2').lemmas():
    pertainyms=pertainyms+lemma.pertainyms()
    antonyms=antonyms+lemma.antonyms()
    deriv=deriv+lemma.derivationally_related_forms()
    print(lemma, 'pertainyms',lemma.pertainyms(),'antonyms',lemma.antonyms(),
          'deriv',lemma.derivationally_related_forms())
    print(lemma.name())

antonyms=deriv[:2]    
print(pertainyms)
print(antonyms)
print(deriv)
print(len(set(antonyms)&set(deriv)))

Lemma('good.n.02.good') pertainyms [] antonyms [Lemma('evil.n.03.evil')] deriv []
good
Lemma('good.n.02.goodness') pertainyms [] antonyms [Lemma('evil.n.03.evilness')] deriv [Lemma('good.s.07.good'), Lemma('estimable.s.02.good'), Lemma('good.a.03.good')]
goodness
[]
[Lemma('good.s.07.good'), Lemma('estimable.s.02.good')]
[Lemma('good.s.07.good'), Lemma('estimable.s.02.good'), Lemma('good.a.03.good')]
1


In [11]:
#analsis the data
n_merged_len=len(df_pairs[(df_pairs['Merge']=='merged')&(df_pairs['Pos']=='n')])
v_merged_len=len(df_pairs[(df_pairs['Merge']=='merged')&(df_pairs['Pos']=='v')])
n_total_len=len(df_pairs[df_pairs['Pos']=='n'])
v_total_len=len(df_pairs[df_pairs['Pos']=='v'])
print('n total {}, n merged {}, v total {}, v merged {}'.format(n_total_len,n_merged_len,
                                                              v_total_len,v_merged_len))
print(n_merged_len/n_total_len,v_merged_len/v_total_len)

n total 2587, n merged 283, v total 10327, v merged 2537
0.1093931194433707 0.24566669894451437


In [21]:
str(df_tmp1[(df_tmp1['WordPos']=='age-v')&(df_tmp1['GroupSense']==2)]['SenseNums'])

'Series([], Name: SenseNums, dtype: object)'

In [35]:
#df_tmp1.drop(columns='Unnamed: 0',inplace=True)


for i in range(len(df_tmp2)):
    Version = df_tmp2.loc[df_tmp2.index[i],'Version']
    WordPos = df_tmp2.loc[df_tmp2.index[i],'WordPos']
    SenseNums = df_tmp2.loc[df_tmp2.index[i],'SenseNums']
    Group = df_tmp2.loc[df_tmp2.index[i],'Group']
    GroupSense = df_tmp2.loc[df_tmp2.index[i],'GroupSense']
    if (',' in SenseNums):
        senses=SenseNums.split(',')
        for s in senses:
            df_tmp1=df_tmp1.append({'Version':Version,'WordPos':WordPos,'SenseNums':s,
                                   'Group':Group,'GroupSense':GroupSense},ignore_index=True)
    elif (' ' in SenseNums):
            senses=SenseNums.split(' ')
            for s in senses:
                df_tmp1=df_tmp1.append({'Version':Version,'WordPos':WordPos,'SenseNums':s,
                                       'Group':Group,'GroupSense':GroupSense},ignore_index=True)
    elif ('.' in SenseNums):
            senses=SenseNums.split('.')
            for s in senses:
                df_tmp1=df_tmp1.append({'Version':Version,'WordPos':WordPos,'SenseNums':s,
                                       'Group':Group,'GroupSense':GroupSense},ignore_index=True)



df_tmp1   

,Version,WordPos,SenseNums,Group,GroupSense
0,2.1,partnership-n,2,1,2
1,2.1,network-n,1,1,1
2,2.1,network-n,2,1,2
3,2.1,network-n,3,1,4
4,2.1,wonder-v,3,1,2
...,...,...,...,...,...
3662,2.1,pay-v,11,1,1
3663,2.1,pay-v,2,1,2
3664,2.1,pay-v,9,1,2
3665,2.1,pay-v,6,1,3


In [107]:
#network#n#1
#network#n#2
for synset in wn2.synsets("network"):
    print(synset)

Synset('network.n.01')
Synset('network.n.02')
Synset('net.n.06')
Synset('network.n.04')
Synset('network.n.05')
Synset('network.v.01')


In [15]:
#remove noisy data (bank-n 2&&3; drug-n 1&&2)
df_note_clean = df_ontonotes[~(df_ontonotes['SenseNum'].str.len()>3)]

df_note_clean=df_note_clean[df_note_clean['Version']=='2.1']
#generate sense pairs
wordpos_list=list(set(df_note_clean['WordPos'].to_list()))

df_note_pairs_21=pd.DataFrame(columns=['Pos','Sense1','Sense2','Merge'])

for w in wordpos_list:
    df_word = df_note_clean[df_note_clean['WordPos']==w]
    for i in range(len(df_word)):
        pos =w.split('-')[1]
        wp_i=df_word.loc[df_word.index[i],'WordPos']
        sn_i=df_word.loc[df_word.index[i],'SenseNum']
        gp_i=df_word.loc[df_word.index[i],'Group']
        sense1=wp_i+'-'+sn_i

        for j in range(i+1,len(df_word)):
            wp_j=df_word.loc[df_word.index[j],'WordPos']
            sn_j=df_word.loc[df_word.index[j],'SenseNum']
            gp_j=df_word.loc[df_word.index[j],'Group']
            sense2 = wp_j+'-'+sn_j

            if gp_i==gp_j:
                
                df_note_pairs_21=df_note_pairs_21.append({'Pos':pos,'Sense1':sense1,'Sense2':sense2,'Merge':'merged'},
                                                  ignore_index=True)
            else:
                
                df_note_pairs_21=df_note_pairs_21.append({'Pos':pos,'Sense1':sense1,'Sense2':sense2,'Merge':'not-merged'},
                                                  ignore_index=True)



In [32]:
len(df_tmp1)

1354

In [37]:
len(df_ontonotes)
len(set(df_ontonotes['WordPos'].to_list()))
a=list(set(df_ontonotes['WordPos'].to_list()))
tl=[]

for i in a:
    tl.append(i+'.xml')
set(tl)
    


{'bring-v.xml',
 'scare-v.xml',
 'happen-v.xml',
 'finance-n.xml',
 'routine-n.xml',
 'examination-n.xml',
 'diminution-n.xml',
 'average-v.xml',
 'degradation-n.xml',
 'state-v.xml',
 'weapon-n.xml',
 'trade-n.xml',
 'shift-v.xml',
 'walk-v.xml',
 'contain-v.xml',
 'entrance-n.xml',
 'knock-v.xml',
 'strategy-n.xml',
 'seem-v.xml',
 'conclude-v.xml',
 'envoy-n.xml',
 'decide-v.xml',
 'trauma-n.xml',
 'slice-n.xml',
 'wait-v.xml',
 'obtain-v.xml',
 'issue-v.xml',
 'yield-v.xml',
 'regard-v.xml',
 'agent-n.xml',
 'impose-v.xml',
 'found-v.xml',
 'treat-n.xml',
 'ingest-v.xml',
 'bloc-n.xml',
 'wing-n.xml',
 'batch-n.xml',
 'allege-v.xml',
 'amusement-n.xml',
 'account-v.xml',
 'peace-n.xml',
 'operate-v.xml',
 'consequence-n.xml',
 'component-n.xml',
 'consult-v.xml',
 'oust-v.xml',
 'facade-n.xml',
 'measure-v.xml',
 'miss-v.xml',
 'lot-n.xml',
 'pass-vpc-v.xml',
 'stick-v.xml',
 'grant-v.xml',
 'agree-v.xml',
 'expand-v.xml',
 'increase-v.xml',
 'mean-v.xml',
 'invite-v.xml',
 'employ

In [40]:
len(set(filelist))
set(tl) - set(filelist)
#set(filelist) - set(tl) 

set()

In [28]:
df_ontonotes[df_ontonotes['WordPos']=='-v']


,WordPos,SenseNum,Group
906,-v,1,1
907,-v,2,1
2872,-v,1,1
2873,-v,2,1
2874,-v,3,1
2875,-v,4,1
2876,-v,5,1


In [58]:
DOMTree = xml.dom.minidom.parse(filepath+'bank-n.xml')
collection = DOMTree.documentElement


if collection.hasAttribute("lemma"):
    wordpos = collection.getAttribute("lemma")
    print("Root element : %s" % collection.getAttribute("lemma"))
    

# 在集合中获取所有sense
senses = collection.getElementsByTagName("sense")

# 打印每部sense的详细信息
for sense in senses:
    if sense.hasAttribute("n"):
        sensenum=sense.getAttribute("n")
        print("n: %s" % sense.getAttribute("n"))
    if sense.hasAttribute("group"):
        group = sense.getAttribute("group")
        print("group: %s" % sense.getAttribute("group"))
    mapping=sense.getElementsByTagName('mappings')
    for mp in mapping:
        wn=mp.getElementsByTagName('wn')
        
        print('sensenum: %s' %wn[0].childNodes[0].data.replace('\n\t','').strip())
        for wnn in wn:
            if wnn.hasAttribute("version"):
                version=wnn.getAttribute("version")
                print('version:',version)

                

    



    

Root element : bank-n
n: 1
group: 1
sensenum: 1
version: 2.0
n: 2
group: 1
sensenum: 4
version: 2.0
n: 2&&3
group: 1
sensenum: -
version: 2.0
n: 3
group: 2
sensenum: 2 9
version: 2.0
n: 4
group: 1
sensenum: 3
version: 2.0
n: 5
group: 1
sensenum: 6
version: 2.0
n: 6
group: 1
sensenum: 5
version: 2.0
n: 7
group: 1
sensenum: 8
version: 2.0
n: 8
group: 2
sensenum: 7
version: 2.0
n: 9
group: 2
sensenum: 10
version: 2.0


In [53]:
len(sensenum.childNodes[0].data)
sensenum.childNodes[0].data.replace('\n\t','').strip()

'10'

In [19]:
from nltk.corpus import wordnet as wn
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')
dog.path_similarity(cat) #synset1.path_similarity(synset2)
dog.lch_similarity(cat) #synset1.lch_similarity(synset2)
dog.wup_similarity(cat)#synset1.wup_similarity(synset2)

from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

dog.res_similarity(cat, brown_ic) #synset1.res_similarity(synset2, ic)
dog.jcn_similarity(cat, brown_ic) #synset1.jcn_similarity(synset2, ic)
dog.lin_similarity(cat, semcor_ic)#synset1.lin_similarity(synset2, ic)

2.0281482472922856

In [20]:
dog.wup_similarity(cat)

0.8571428571428571

In [21]:
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

In [25]:
dog.res_similarity(cat, brown_ic) #synset1.res_similarity(synset2, ic)
dog.jcn_similarity(cat, brown_ic) #synset1.jcn_similarity(synset2, ic)
dog.lin_similarity(cat, semcor_ic)#synset1.lin_similarity(synset2, ic)

0.8863288628086228

In [33]:
#wordnet 2.1
import nltk
from nltk.corpus import WordNetCorpusReader
wn2 = WordNetCorpusReader("/Users/gary/Documents/perl/package/WordNet-2.1/dict", nltk.data.find("/Users/gary/Documents/perl/package/WordNet-2.1/dict"))
print(wn2.get_version())
for synset in wn2.synsets('bank'):
    print(synset)

2.1
Synset('depository_financial_institution.n.01')
Synset('bank.n.02')
Synset('bank.n.03')
Synset('bank.n.04')
Synset('bank.n.05')
Synset('savings_bank.n.02')
Synset('bank.n.07')
Synset('bank.n.08')
Synset('bank.n.09')
Synset('bank.n.10')
Synset('bank.v.01')
Synset('bank.v.02')
Synset('bank.v.03')
Synset('bank.v.04')
Synset('bank.v.05')
Synset('deposit.v.02')
Synset('bank.v.07')
Synset('trust.v.01')


In [41]:

for synset in wn2.synsets('bank','n'):
    print(synset, synset.definition())

Synset('depository_financial_institution.n.01') a financial institution that accepts deposits and channels the money into lending activities
Synset('bank.n.02') sloping land (especially the slope beside a body of water)
Synset('bank.n.03') a supply or stock held in reserve for future use (especially in emergencies)
Synset('bank.n.04') a building in which the business of banking transacted
Synset('bank.n.05') an arrangement of similar objects in a row or in tiers
Synset('savings_bank.n.02') a container (usually with a slot in the top) for keeping money at home
Synset('bank.n.07') a long ridge or pile
Synset('bank.n.08') the funds held by a gambling house or the dealer in some gambling games
Synset('bank.n.09') a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force
Synset('bank.n.10') a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)


In [38]:
print(wn.get_version())
for synset in wn.synsets('bank'):
    print(synset,synset.definition())

3.0
Synset('bank.n.01') sloping land (especially the slope beside a body of water)
Synset('depository_financial_institution.n.01') a financial institution that accepts deposits and channels the money into lending activities
Synset('bank.n.03') a long ridge or pile
Synset('bank.n.04') an arrangement of similar objects in a row or in tiers
Synset('bank.n.05') a supply or stock held in reserve for future use (especially in emergencies)
Synset('bank.n.06') the funds held by a gambling house or the dealer in some gambling games
Synset('bank.n.07') a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force
Synset('savings_bank.n.02') a container (usually with a slot in the top) for keeping money at home
Synset('bank.n.09') a building in which the business of banking transacted
Synset('bank.n.10') a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)
Synset('bank.v.01') tip laterall

In [7]:
15800/(11*52)

27.622377622377623

In [6]:
13024+17500+15800+6765-8483

44606

In [150]:
#concatnate all the features together.

#'note_pairs_wnsimilarity.csv' # wordnet::similarity package
features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
feature_files=['OntoNotes_SensesPairs.csv','note_pairs_wnsimilarity.csv','OntoNotes_SensesPairs_WNFeatures.csv',
              'WN21mapWn16_topic_similarity.csv','WN21mapWn16_WN_Domain_feature.csv','WN_OED_Map_Feature_OntoNotes_SensesPairs.csv']

df_sensepari=pd.read_csv(features_path+feature_files[0])
df_sensepari= df_sensepari[['Pos', 'Sense1', 'Sense2', 'Merge']]
df_sensepari=df_sensepari.drop_duplicates()

print('df_sensepari',len(df_sensepari))
df_wnpackage=pd.read_csv(features_path+feature_files[1])
print('df_wnpackage',len(df_wnpackage))

df_features_tmp = df_sensepari.merge(df_wnpackage,left_on=['Sense1', 'Sense2','Merge'], 
                              right_on=['sense1', 'sense2','merge'], how='left')


df_features_tmp=df_features_tmp[['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue',
       'wupvalue']]
df_features_tmp=df_features_tmp.drop_duplicates()
print('df_features_tmp',len(df_features_tmp))

df_wncorpus=pd.read_csv(features_path+feature_files[2])

print('df_wncorpus',len(df_wncorpus))

df_features_tmp1 = df_features_tmp.merge(df_wncorpus,left_on=['Pos', 'Sense1', 'Sense2', 'Merge'], 
                              right_on=['Pos', 'Sense1', 'Sense2', 'Merge'], how='left')

df_features_tmp1.columns
print('df_features_tmp1',len(df_features_tmp1))


df_features_tmp1=df_features_tmp1[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 
        'pertainyms', 'antonyms', 'deriv', 'lemmas',
       'verbgroup', 'verbframe', 'hyper_min', 'hyper_max']]

df_features_tmp1=df_features_tmp1.drop_duplicates() 

#topic feature
df_topfea=pd.read_csv(features_path+feature_files[3])
#replace '.' with '#'
df_topfea['sense1_wn21']=df_topfea['sense1_wn21'].str.replace('.','#')
df_topfea['sense2_wn21']=df_topfea['sense2_wn21'].str.replace('.','#')

df_topfea
print('df_topfea',len(df_topfea))

df_features_tmp2 = df_features_tmp1.merge(df_topfea[['pos','sense1_wn21','sense2_wn21','topic_similarity']],left_on=['Pos', 'Sense1', 'Sense2'], 
                              right_on=['pos', 'sense1_wn21','sense2_wn21'], how='left')


print('df_features_tmp2',len(df_features_tmp2))

df_features_tmp2=df_features_tmp2[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'pertainyms',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe', 'hyper_min',
       'hyper_max', 'topic_similarity']]

df_features_tmp2=df_features_tmp2.drop_duplicates()
#domain feature
df_domainfea=pd.read_csv(features_path+feature_files[4])

df_domainfea['sense1_wn21']=df_domainfea['sense1_wn21'].str.replace('.','#')
df_domainfea['sense2_wn21']=df_domainfea['sense2_wn21'].str.replace('.','#')
print('df_domainfea',len(df_domainfea))

df_features_tmp3 = df_features_tmp2.merge(df_domainfea[['pos','sense1_wn21','sense2_wn21','wn_domain_feature']],left_on=['Pos', 'Sense1', 'Sense2'], 
                              right_on=['pos', 'sense1_wn21','sense2_wn21'], how='left')

df_features_tmp3.columns
print('df_features_tmp3',len(df_features_tmp3))

df_features_tmp3=df_features_tmp3[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'pertainyms',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe', 'hyper_min',
       'hyper_max', 'topic_similarity', 'wn_domain_feature']]

df_features_tmp3=df_features_tmp3.drop_duplicates()
#wn-oed mapping feature
df_wnoedfea=pd.read_csv(features_path+feature_files[5])
df_wnoedfea
print('df_wnoedfea',len(df_wnoedfea))
df_features = df_features_tmp3.merge(df_wnoedfea[['Pos', 'Sense1', 'Sense2', 'Merge','wn_oed_feature']],left_on=['Pos', 'Sense1', 'Sense2','Merge'], 
                              right_on=['Pos', 'Sense1', 'Sense2', 'Merge'], how='left')

df_features=df_features[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'pertainyms',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe', 'hyper_min',
       'hyper_max', 'topic_similarity', 'wn_domain_feature', 'wn_oed_feature']]

df_features=df_features.drop_duplicates()
df_features
print('df_features',len(df_features))

#since NaN, there are still duplicated rows in df_features. drop duplicates after replacing
#Nan with other values


#update verb group using results from perl
#use perl to get the following features: antonyms, deriv, pertainyms, verbgroups
file_path='/Users/gary/Documents/2020Fall/IntroNLP/project/note_pairs_wncourpus_similarity.csv'
df_perl=pd.read_csv(file_path)

df_perl_vg=df_perl[df_perl[' vgrp1']==df_perl[' vgrp2']]
for i in range(len(df_perl_vg)):
    sense1=df_perl_vg.loc[df_perl_vg.index[i],'sense1']
    sense2=df_perl_vg.loc[df_perl_vg.index[i],' sense2']
    df_features.loc[((df_features['Sense1']==sense1)&(df_features['Sense2']==sense2)),'verbgroup']=1


df_features.to_csv('feature_space.csv',index=False)



df_sensepari 12902
df_wnpackage 12915
df_features_tmp 12911
df_wncorpus 12914
df_features_tmp1 12932
df_topfea 1333
df_features_tmp2 12923
df_domainfea 12334
df_features_tmp3 12934
df_wnoedfea 12458
df_features 12911


In [3]:
#analyze the feature space
features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
df_features= pd.read_csv(features_path+'feature_space.csv')
#remove wrong senses (no sense number)
df_features = df_features[df_features['Sense1'].str.contains('[0-9]')]
df_features = df_features[df_features['Sense2'].str.contains('[0-9]')]
df_features
#df_features[~(df_features['Sense1'].str.contains('[0-9]'))]
#df_features[~(df_features['Sense2'].str.contains('[0-9]'))]

#deal with NaN columns
#remove 'lch' is NaN rows, as all other features are also NaN
df_features = df_features[~(df_features['lch'].isna())]
# hso, jcn, leskvalue, linvalue, resvalue,vecvalue, wupvalue, no NaN
#pertainyms,antonyms, deriv,lemmas  no NaN

#verbgroup, verbframe have no NaN
df_features[df_features['verbgroup'].isna()]

df_features[df_features['verbgroup']==1]
df_features.to_csv('feature_space.csv',index=False)

In [4]:
df_features[df_features['verbgroup']==1]

,Unnamed: 0,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,...,antonyms,deriv,lemmas,verbgroup,verbframe,hyper_min,hyper_max,topic_similarity,wn_domain_feature,wn_oed_feature
243,257,v,feel#v#10,feel#v#13,merged,2.233592,5.0,0.000000,68.0,0.000000,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,1.0,1.0
1970,2038,v,convert#v#6,convert#v#7,merged,2.233592,5.0,0.000000,28.0,0.000000,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,1.0,1.0
2949,3108,v,kill#v#1,kill#v#10,merged,2.233592,0.0,0.000000,16.0,0.000000,...,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0
4855,5127,v,play#v#1,play#v#31,merged,1.386294,0.0,0.000000,23.0,0.000000,...,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0
4935,5207,v,play#v#32,play#v#33,merged,2.233592,5.0,0.000000,376.0,0.000000,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,1.0,1.0
5183,5455,v,play#v#14,play#v#26,merged,1.540445,0.0,0.043985,39.0,0.000000,...,0.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,1.0,1.0
5216,5488,v,play#v#10,play#v#30,merged,2.233592,5.0,0.706981,40.0,0.928825,...,0.0,1.0,1.0,1.0,1.0,0.0,2.0,NaN,1.0,1.0
5769,6056,v,fail#v#6,fail#v#8,merged,1.722767,0.0,0.000000,3.0,0.000000,...,0.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,1.0,1.0
7515,7875,v,read#v#10,read#v#5,not-merged,1.134980,0.0,0.000000,8.0,0.000000,...,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0
7814,8197,v,tap#v#10,tap#v#11,merged,1.252763,0.0,0.000000,17.0,0.000000,...,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0


In [164]:
df_features.info()
df_features.describe()
#df_features.loc[:,df_features.isna().any()]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12333 entries, 0 to 12909
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         12333 non-null  int64  
 1   Pos                12333 non-null  object 
 2   Sense1             12333 non-null  object 
 3   Sense2             12333 non-null  object 
 4   Merge              12333 non-null  object 
 5   lch                12331 non-null  float64
 6   hso                12332 non-null  float64
 7   jcn                12331 non-null  float64
 8   leskvalue          12331 non-null  float64
 9   linvalue           12331 non-null  float64
 10  resvalue           12331 non-null  float64
 11  vecvalue           12332 non-null  float64
 12  wupvalue           12331 non-null  float64
 13  pertainyms         12331 non-null  float64
 14  antonyms           12331 non-null  float64
 15  deriv              12331 non-null  float64
 16  lemmas             123

,Unnamed: 0,lch,hso,jcn,leskvalue,linvalue,resvalue,vecvalue,wupvalue,pertainyms,antonyms,deriv,lemmas,verbgroup,verbframe,hyper_min,hyper_max,topic_similarity,wn_domain_feature,wn_oed_feature
count,12333.000000,12331.000000,12332.000000,1.233100e+04,12331.000000,12331.000000,12331.000000,12332.000000,12331.000000,12331.0,12331.000000,12331.000000,12331.000000,9800.0,9800.000000,3473.000000,3473.000000,1331.000000,12331.0,12333.000000
mean,6468.069407,1.432354,0.522867,5.409967e+03,18.724434,0.041080,0.609479,0.503000,0.327255,0.0,0.002919,0.284730,1.052713,0.0,0.517245,3.698388,5.205298,0.805577,1.0,0.668694
std,3729.849550,0.393391,1.923029,3.639528e+05,41.170958,0.138674,1.537815,0.194532,0.153645,0.0,0.053955,0.451304,0.264366,0.0,0.499728,2.588487,2.712996,0.232458,0.0,0.470702
min,0.000000,0.287682,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.090909,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000
25%,3248.000000,1.134980,0.000000,0.000000e+00,5.000000,0.000000,0.000000,0.353277,0.222222,0.0,0.000000,0.000000,1.000000,0.0,0.000000,1.500000,3.000000,0.773667,1.0,0.000000
50%,6486.000000,1.386294,0.000000,5.024623e-02,10.000000,0.000000,0.000000,0.496569,0.285714,0.0,0.000000,0.000000,1.000000,0.0,1.000000,3.500000,6.000000,0.884335,1.0,1.000000
75%,9686.000000,1.722767,0.000000,6.004106e-02,18.000000,0.000000,0.000000,0.637217,0.400000,0.0,0.000000,1.000000,1.000000,0.0,1.000000,5.500000,7.000000,0.939164,1.0,1.000000
max,12930.000000,3.637586,16.000000,2.724020e+07,1336.000000,1.000000,12.515034,1.000000,1.000000,0.0,1.000000,1.000000,4.000000,0.0,1.000000,11.500000,13.000000,1.000000,1.0,1.000000


In [5]:
df_features.loc[:,df_features.isna().any()]
df_features.columns[df_features.isna().any()].to_list()

['verbgroup', 'verbframe', 'hyper_min', 'hyper_max', 'topic_similarity']

In [6]:
#for each PoS, split the features into a training set (70%) and a testing set (30%)
# I remove wn_domain_feature,as it is same (1) for all pairs
# I remove 'hyper_min', 'hyper_max', 'topic_similarity' as they have NaN values 
# I remove 'pertainyms' as it is for adj.
features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
df_features= pd.read_csv(features_path+'feature_space.csv')

df_features = df_features[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn',
       'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'pertainyms', 'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe',
       'hyper_min', 'hyper_max', 'topic_similarity', 'wn_domain_feature',
       'wn_oed_feature']]
df_features_n = df_features[df_features['Pos']=='n'][['Pos', 'Sense1', 'Sense2', 'Merge', 
        'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas','wn_oed_feature']]
df_features_v = df_features[df_features['Pos']=='v'][['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn','leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe','wn_oed_feature']]


In [7]:
df_features_v.columns[df_features_v.isna().any()].to_list()
df_features_v.describe()

,lch,hso,jcn,leskvalue,linvalue,resvalue,vecvalue,wupvalue,antonyms,deriv,lemmas,verbgroup,verbframe,wn_oed_feature
count,9800.000000,9800.000000,9.800000e+03,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000
mean,1.490063,0.546224,1.247940e+03,15.485102,0.034659,0.456952,0.512372,0.306312,0.003469,0.300510,1.060306,0.002041,0.517245,0.694592
std,0.366383,2.024320,1.235346e+05,33.891906,0.138743,1.458769,0.194091,0.135121,0.058802,0.458503,0.265236,0.045132,0.499728,0.460604
min,0.287682,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.028795,0.090909,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,1.252763,0.000000,0.000000e+00,5.000000,0.000000,0.000000,0.362942,0.222222,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,1.540445,0.000000,5.123915e-02,9.000000,0.000000,0.000000,0.505379,0.285714,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
75%,1.722767,0.000000,6.047147e-02,15.000000,0.000000,0.000000,0.643684,0.333333,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,2.639057,16.000000,1.222930e+07,1336.000000,1.000000,11.714175,0.996472,0.933333,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000


In [8]:
#replace labels with 0 or 1
df_features_v=df_features_v.replace(['not-merged','merged'],[0,1])
v_nm=len(df_features_v[df_features_v['Merge']==0])
v_m=len(df_features_v[df_features_v['Merge']==1])
print('v->not_merged:{}, merged:{}'.format(v_nm,v_m))

df_features_n=df_features_n.replace(['not-merged','merged'],[0,1])
n_nm=len(df_features_n[df_features_n['Merge']=='not-merged'])
n_m=len(df_features_n[df_features_n['Merge']=='merged'])
print('n->not_merged:{}, merged:{}'.format(n_nm,n_m))
df_features_v

v->not_merged:7263, merged:2537
n->not_merged:2249, merged:282


,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,resvalue,vecvalue,wupvalue,antonyms,deriv,lemmas,verbgroup,verbframe,wn_oed_feature
0,v,consult#v#3,consult#v#4,not-merged,1.945910,4.0,0.000000,1.0,0.000000,5.909040,0.192654,0.823529,0.0,1.0,1.0,0.0,1.0,1.0
1,v,consult#v#3,consult#v#1,not-merged,1.386294,0.0,0.073668,5.0,0.301704,2.932466,0.458805,0.571429,0.0,1.0,1.0,0.0,1.0,1.0
2,v,consult#v#3,consult#v#2,not-merged,0.847298,0.0,0.051132,7.0,0.000000,0.000000,0.358709,0.153846,0.0,1.0,1.0,0.0,0.0,1.0
3,v,consult#v#4,consult#v#1,not-merged,1.252763,0.0,0.000000,2.0,0.000000,2.932466,0.255969,0.533333,0.0,1.0,1.0,0.0,1.0,1.0
4,v,consult#v#4,consult#v#2,not-merged,0.767255,0.0,0.000000,3.0,0.000000,0.000000,0.238449,0.142857,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,v,discover#v#4,discover#v#7,merged,1.945910,0.0,0.054114,13.0,0.000000,0.000000,0.399127,0.400000,0.0,1.0,1.0,0.0,0.0,1.0
12326,v,discover#v#4,discover#v#8,merged,1.945910,0.0,0.057534,3.0,0.000000,0.000000,0.435737,0.400000,0.0,0.0,1.0,0.0,0.0,1.0
12327,v,discover#v#5,discover#v#7,merged,1.722767,0.0,0.051853,4.0,0.000000,0.000000,0.286203,0.333333,0.0,1.0,1.0,0.0,1.0,1.0
12328,v,discover#v#5,discover#v#8,merged,1.722767,0.0,0.054986,1.0,0.000000,0.000000,0.194024,0.333333,0.0,0.0,1.0,0.0,1.0,1.0


In [12]:
df_features_v 

,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,resvalue,vecvalue,wupvalue,antonyms,deriv,lemmas,verbgroup,verbframe,wn_oed_feature
0,v,consult#v#3,consult#v#4,0,1.945910,4.0,0.000000,1.0,0.000000,5.909040,0.192654,0.823529,0.0,1.0,1.0,0.0,1.0,1.0
1,v,consult#v#3,consult#v#1,0,1.386294,0.0,0.073668,5.0,0.301704,2.932466,0.458805,0.571429,0.0,1.0,1.0,0.0,1.0,1.0
2,v,consult#v#3,consult#v#2,0,0.847298,0.0,0.051132,7.0,0.000000,0.000000,0.358709,0.153846,0.0,1.0,1.0,0.0,0.0,1.0
3,v,consult#v#4,consult#v#1,0,1.252763,0.0,0.000000,2.0,0.000000,2.932466,0.255969,0.533333,0.0,1.0,1.0,0.0,1.0,1.0
4,v,consult#v#4,consult#v#2,0,0.767255,0.0,0.000000,3.0,0.000000,0.000000,0.238449,0.142857,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,v,discover#v#4,discover#v#7,1,1.945910,0.0,0.054114,13.0,0.000000,0.000000,0.399127,0.400000,0.0,1.0,1.0,0.0,0.0,1.0
12326,v,discover#v#4,discover#v#8,1,1.945910,0.0,0.057534,3.0,0.000000,0.000000,0.435737,0.400000,0.0,0.0,1.0,0.0,0.0,1.0
12327,v,discover#v#5,discover#v#7,1,1.722767,0.0,0.051853,4.0,0.000000,0.000000,0.286203,0.333333,0.0,1.0,1.0,0.0,1.0,1.0
12328,v,discover#v#5,discover#v#8,1,1.722767,0.0,0.054986,1.0,0.000000,0.000000,0.194024,0.333333,0.0,0.0,1.0,0.0,1.0,1.0


In [15]:
#apply SVM for Verb F1=0.4989
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,StratifiedKFold,LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
import numpy as np

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

def split_data(X,Y,rate):
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=rate,random_state=42)
    return X_train,X_test,Y_train,Y_test

def cross_validation(X_train, Y_train, C_list, K):
    for c in C_list:
        clf1 = SVC(kernel='linear',C=c).fit(X_train,Y_train)

        strat_scores = cross_val_score(clf1,X_train,Y_train,cv=StratifiedKFold(K,random_state=10,shuffle=True))
        print('C=',c)
        print("The Stratified Cross Validation Score :"+str(strat_scores))
        print("The Average Stratified Cross Validation Score :"+str(strat_scores.mean()))

def predict(X_train, Y_train, X_test, c):
    clf1 = SVC(kernel='linear', C=c).fit(X_train,Y_train)
    #print('features important:')
    #f_importances(clf1.coef_, features_names)
    print("shape of Y :"+str(Y_train.shape))
    print("shape of X :"+str(X_train.shape))
    y_pred = clf1.predict(X_test)
    return clf1, y_pred

def cal_FScore(y_true, y_pred):
    F1=f1_score(y_true, y_pred, average='macro')
    return F1


#for each PoS, split the features into a training set (70%) and a testing set (30%)
# I remove wn_domain_feature,as it is same (1) for all pairs
# I remove 'hyper_min', 'hyper_max', 'topic_similarity' as they have NaN values 
# I remove 'pertainyms' as it is for adj.
features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
df_features= pd.read_csv(features_path+'feature_space.csv')

df_features = df_features[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn',
       'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'pertainyms', 'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe',
       'hyper_min', 'hyper_max', 'topic_similarity', 'wn_domain_feature',
       'wn_oed_feature']]
df_features_n = df_features[df_features['Pos']=='n'][['Pos', 'Sense1', 'Sense2', 'Merge', 
        'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas','wn_oed_feature']]
df_features_v = df_features[df_features['Pos']=='v'][['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn','leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe','wn_oed_feature']]

#replace labels with 0 or 1
df_features_v=df_features_v.replace(['not-merged','merged'],[0,1])
v_nm=len(df_features_v[df_features_v['Merge']==0])
v_m=len(df_features_v[df_features_v['Merge']==1])
print('v->not_merged:{}, merged:{}'.format(v_nm,v_m))

df_features_n=df_features_n.replace(['not-merged','merged'],[0,1])
n_nm=len(df_features_n[df_features_n['Merge']==0])
n_m=len(df_features_n[df_features_n['Merge']==1])
print('n->not_merged:{}, merged:{}'.format(n_nm,n_m))
#df_features_v


Y_v = np.array(df_features_v['Merge'])
X_v = df_features_v.drop(['Pos','Sense1','Sense2','Merge'],axis=1)
X_v = np.array(X_v)
features_names=['1','2','3','4','5','6','7','8','9','10','11','12','13','14']
print("shape of Y :"+str(Y_v.shape))
print("shape of X :"+str(X_v.shape))

#normalize data, Snow did not mention this
scaler = MinMaxScaler()
X_v = scaler.fit_transform(X_v)

#split data
rate=0.3
X_train,X_test,Y_train,Y_test = split_data(X_v,Y_v,rate)

#cross validation c=3 average score=0.75233236151603
C_list=[1,2,3,5,10]
K=10
#cross_validation(X_train, Y_train, C_list, K)
c=3

#predict
clf1, y_pred= predict(X_train, Y_train, X_test, c)

#F1 score
F1= cal_FScore(Y_test,y_pred)

print('Verb:',F1)
#base line
y_pre_baseline_v=np.ones(len(Y_test))
F1= cal_FScore(Y_test,y_pre_baseline_v)

print('Baseline:',F1)



v->not_merged:7263, merged:2537
n->not_merged:2249, merged:282
shape of Y :(9800,)
shape of X :(9800, 14)
shape of Y :(6860,)
shape of X :(6860, 14)
Verb: 0.4989548893582957
Baseline: 0.20454545454545453


In [3]:
clf1.coef_
#Verb: 0.4995113221832198
df_features_n

,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,resvalue,vecvalue,wupvalue,antonyms,deriv,lemmas,wn_oed_feature
9,n,chain#n#1,chain#n#2,0,1.152680,0.0,0.046965,53.0,0.000000,0.000000,0.660349,0.266667,0.0,1.0,1.0,1.0
10,n,chain#n#1,chain#n#4,0,1.335001,0.0,0.056717,51.0,0.212567,2.379800,0.690059,0.526316,0.0,0.0,1.0,1.0
11,n,chain#n#1,chain#n#9,0,0.804373,0.0,0.044661,22.0,0.000000,0.000000,0.665821,0.200000,0.0,0.0,1.0,1.0
12,n,chain#n#1,chain#n#8,0,1.152680,0.0,0.044661,67.0,0.000000,0.000000,0.563954,0.266667,0.0,0.0,1.0,1.0
13,n,chain#n#1,chain#n#6,0,1.335001,0.0,0.000000,34.0,0.000000,0.516189,0.627652,0.470588,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12288,n,deposit#n#8,deposit#n#7,0,0.929536,0.0,0.000000,22.0,0.000000,0.000000,0.800074,0.222222,0.0,0.0,1.0,1.0
12289,n,deposit#n#9,deposit#n#6,0,0.864997,0.0,0.000000,4.0,0.000000,0.516189,0.522108,0.347826,0.0,0.0,1.0,1.0
12290,n,deposit#n#9,deposit#n#7,0,0.804373,0.0,0.000000,3.0,0.000000,0.516189,0.541884,0.333333,0.0,0.0,1.0,1.0
12291,n,deposit#n#6,deposit#n#7,1,1.152680,0.0,0.000000,9.0,0.000000,0.516189,0.673918,0.421053,0.0,0.0,1.0,1.0


In [7]:
df_features_v

,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,resvalue,vecvalue,wupvalue,antonyms,deriv,lemmas,verbgroup,verbframe,wn_oed_feature
0,v,consult#v#3,consult#v#4,0,1.945910,4.0,0.000000,1.0,0.000000,5.909040,0.192654,0.823529,0.0,1.0,1.0,0.0,1.0,1.0
1,v,consult#v#3,consult#v#1,0,1.386294,0.0,0.073668,5.0,0.301704,2.932466,0.458805,0.571429,0.0,1.0,1.0,0.0,1.0,1.0
2,v,consult#v#3,consult#v#2,0,0.847298,0.0,0.051132,7.0,0.000000,0.000000,0.358709,0.153846,0.0,1.0,1.0,0.0,0.0,1.0
3,v,consult#v#4,consult#v#1,0,1.252763,0.0,0.000000,2.0,0.000000,2.932466,0.255969,0.533333,0.0,1.0,1.0,0.0,1.0,1.0
4,v,consult#v#4,consult#v#2,0,0.767255,0.0,0.000000,3.0,0.000000,0.000000,0.238449,0.142857,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,v,discover#v#4,discover#v#7,1,1.945910,0.0,0.054114,13.0,0.000000,0.000000,0.399127,0.400000,0.0,1.0,1.0,0.0,0.0,1.0
12326,v,discover#v#4,discover#v#8,1,1.945910,0.0,0.057534,3.0,0.000000,0.000000,0.435737,0.400000,0.0,0.0,1.0,0.0,0.0,1.0
12327,v,discover#v#5,discover#v#7,1,1.722767,0.0,0.051853,4.0,0.000000,0.000000,0.286203,0.333333,0.0,1.0,1.0,0.0,1.0,1.0
12328,v,discover#v#5,discover#v#8,1,1.722767,0.0,0.054986,1.0,0.000000,0.000000,0.194024,0.333333,0.0,0.0,1.0,0.0,1.0,1.0


In [17]:

#apply SVM for Noun, F1=0.4991
Y_n = np.array(df_features_n['Merge'])
X_n = df_features_n.drop(['Pos','Sense1','Sense2','Merge'],axis=1)
X_n = np.array(X_n)

print("shape of Y :"+str(Y_n.shape))
print("shape of X :"+str(X_n.shape))

#normalize data, Snow did not mention this
scaler = MinMaxScaler()
X_n = scaler.fit_transform(X_n)

#split data
rate=0.3
X_train_n,X_test_n,Y_train_n,Y_test_n = split_data(X_n,Y_n,rate)

#cross validation c=3 average score=0.75233236151603
C_list=[1,2,3,5,10]
K=10
#cross_validation(X_train, Y_train, C_list, K)
c=1
 
#predict
clf2,y_pred_n= predict(X_train_n, Y_train_n, X_test_n, c)

#F1 score
F1= cal_FScore(Y_test_n,y_pred_n)

print('Noun:',F1)

#base line
y_pre_baseline_n=np.ones(len(Y_test_n))
F1= cal_FScore(Y_test_n,y_pre_baseline_n)

print('Baseline:',F1)


shape of Y :(2531,)
shape of X :(2531, 12)
shape of Y :(1771,)
shape of X :(1771, 12)
Noun: 0.4696441032798325
Baseline: 0.10271546635182999


In [94]:
clf1.coef_
#0.4696441032798325

array([[-3.88419461e-04,  3.80488957e-01, -4.27429430e-07,
         1.98804936e-02,  2.28596767e+00, -1.85646511e-01,
         1.22427718e-04,  1.10407741e-03,  1.99853618e+00,
         1.14210661e-06,  2.83184328e-04,  1.99994550e+00,
         1.78414461e-04, -1.67182964e-04]])

In [55]:
X_train,X_test,Y_train,Y_test = train_test_split(X_v,Y_v,test_size=.30,random_state=42)
print("shape of X Train :"+str(X_train.shape))
print("shape of X Test :"+str(X_test.shape))
print("shape of Y Train :"+str(Y_train.shape))
print("shape of Y Test :"+str(Y_test.shape))
''' 
for this_C in [1,3,5,10,40,60,80,100]:
    clf = SVC(kernel='linear',C=this_C).fit(X_train,Y_train)
    scoretrain = clf.score(X_train,Y_train)
    scoretest  = clf.score(X_test,Y_test)
    print("Linear SVM value of C:{}, training score :{:2f} , Test Score: {:2f} \n".format(this_C,scoretrain,scoretest))
    
'''

shape of X Train :(6860, 14)
shape of X Test :(2940, 14)
shape of Y Train :(6860,)
shape of Y Test :(2940,)


' \nfor this_C in [1,3,5,10,40,60,80,100]:\n    clf = SVC(kernel=\'linear\',C=this_C).fit(X_train,Y_train)\n    scoretrain = clf.score(X_train,Y_train)\n    scoretest  = clf.score(X_test,Y_test)\n    print("Linear SVM value of C:{}, training score :{:2f} , Test Score: {:2f} \n".format(this_C,scoretrain,scoretest))\n    \n'

In [25]:
from sklearn.model_selection import cross_val_score,StratifiedKFold,LeaveOneOut
clf1 = SVC(kernel='linear',C=1).fit(X_train,Y_train)
scores = cross_val_score(clf1,X_train,Y_train,cv=10)
strat_scores = cross_val_score(clf1,X_train,Y_train,cv=StratifiedKFold(10,random_state=10,shuffle=True))
#Loo = LeaveOneOut()
#Loo_scores = cross_val_score(clf1,X_train,Y_train,cv=Loo)
print("The Cross Validation Score :"+str(scores))
print("The Average Cross Validation Score :"+str(scores.mean()))
print("The Stratified Cross Validation Score :"+str(strat_scores))
print("The Average Stratified Cross Validation Score :"+str(strat_scores.mean()))

The Cross Validation Score :[0.75364431 0.75510204 0.75947522 0.74635569 0.74927114 0.75510204
 0.74781341 0.75510204 0.74198251 0.75510204]
The Average Cross Validation Score :0.7518950437317784
The Stratified Cross Validation Score :[0.75510204 0.76239067 0.76239067 0.74927114 0.75218659 0.75655977
 0.75364431 0.73760933 0.74489796 0.75072886]
The Average Stratified Cross Validation Score :0.7524781341107871


In [34]:
#print(len(clf1.predict(X_test)),len(Y_test))
from sklearn.metrics import f1_score
y_pred = clf1.predict(X_test)
y_true = Y_test
f1_score(y_true, y_pred, average='macro')

0.49915340683152853

In [32]:
cnt=1
for i in list(clf1.predict(X_test)-Y_test):
    if i==-1:
        cnt+=1
print(cnt,cnt/2940)

ct = 1
for j in Y_test:
    if j==1:
        ct+=1
print(ct, ct/2940)
        

694 0.23605442176870747
757 0.2574829931972789


In [43]:
1214/(11974+1680+4930)

0.06532501076194576

In [44]:
1214/11974

0.10138633706363788